In [1]:
import json
import requests
import os
import pandas as pd
from urllib.parse import quote

In [57]:
file = open('../key.json', 'r')
API_KEY = json.load(file)[0]['OMDB_API']

In [63]:
# movie_dict = pd.read_csv("movies.csv").drop("Unnamed: 0", axis=1)
# movie_dict.to_csv("movies.csv", index=False)

In [118]:
movie_dict = pd.read_csv("movies.csv").to_dict(orient="records")
movies = [i['title'].lower().replace("º", "°").replace("’", "'") for i in movie_dict] #movie list

In [133]:
scrapped = pd.read_csv("new_merged_scrapped.csv")
scrapped = scrapped.drop_duplicates(subset='Title', keep='first')
scrapped_dict = scrapped.to_dict(orient="records")
scrapped_movies = [i['Title'].lower() for i in scrapped_dict]

In [136]:
# "Alexander and the Terrible, Horrible, No Good, …".replace(" …", "")
# "Alexander and the Terrible, Horrible, No Good, Very Bad Day".find("Alexander and the Terrible, Horrible, No Good,")

# l = ["done", "Alexander and the Terrible, Horrible, No Good, Very Bad Day"]
# l[1].endswith(" …")
# scrapped = scrapped.drop("Unnamed: 0.1", axis=1)
# scrapped = scrapped.drop("Unnamed: 0", axis=1)

# scrapped = scrapped.drop(272)
# scrapped = scrapped.drop(743)
# scrapped = scrapped.drop(776)
# scrapped.to_csv("movie_info.csv", index=False)


# "no se aceptan devoluciones 2: el regreso de loreto peralta".find("no se aceptan devoluciones")
# len(scrapped)
scrapped.to_csv('movie_info.csv', index=False)

In [120]:
for index, i in enumerate(movies):
    if i.endswith("…"):
        movies[index] = i.replace("…", "").strip()
        # print(i)

In [129]:
movies = list(set(movies))
scrapped_movies = list(set(scrapped_movies))

In [122]:
leftover = []
for movie in movies:
    if movie not in scrapped_movies:
        leftover.append(movie)

In [123]:
df1 = pd.DataFrame({'movies': movies})
df2 = pd.DataFrame({'leftover': leftover})
df3 = pd.DataFrame({'scrapped':scrapped_movies})

result_df = pd.merge(df1, df2, left_on='movies', right_on='leftover', how='outer')

result_df = pd.merge(result_df, df3, left_on='movies', right_on='scrapped', how='outer')

result_df.to_csv("output.csv")


In [124]:
len(movies), len(scrapped_movies), len(leftover) , (len(scrapped_movies)+len(leftover))

(2005, 1873, 132, 2005)

In [66]:
new_list = []
count = 0
not_added = 0

for m in leftover:
    formatted = quote(m)
    url = f"http://www.omdbapi.com/?t={formatted}&apikey=2c60d316"
    response = requests.get(url)
    # print(url)
    if response.status_code == 200:
        data = json.loads(response.text)
        if data['Response'] != 'False':
            new_list.append(data)
            print(data['Title'], m)
            count += 1
        else:
            not_added += 1
    else:
        print(f"Status code: {response.status_code}")
        break

In [67]:
count , not_added

(0, 132)

In [81]:
new_scrapped = pd.DataFrame(new_list)
len(new_scrapped), len(scrapped), new_scrapped.columns, scrapped.columns

(0,
 1935,
 RangeIndex(start=0, stop=0, step=1),
 Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
        'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
        'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
        'DVD', 'BoxOffice', 'Production', 'Website', 'totalSeasons',
        'Response'],
       dtype='object'))

In [89]:
merged_new = pd.concat([scrapped, new_scrapped], ignore_index = True)

In [90]:
merged_new.to_csv('new_merged_scrapped.csv', index=False)

In [226]:
# dms = pd.read_csv("new_merged_scrapped.csv")
# dms = dms.drop_duplicates()

# dms = dms.drop(791)
# dms.to_csv("new_merged_scrapped.csv", index=False)
# # dms = dms.drop(752)

In [79]:
len(scrapped)

1935

(7, 1928)